# Build Model for Training, and Test accuracy

In [10]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from PIL import Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
import cPickle as pickle
import tensorflow as tf

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline
np.set_printoptions(threshold=np.inf)

### Deserialize

In [11]:
train_pickle_file = 'SVHN_basic_train_labels.pickle'

with open(train_pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_labels = save['train_image_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_labels.shape)
print(train_labels[0:2,:])

Training set (33402, 6)
[[ 2.  0.  0.  0.  1.  9.]
 [ 2.  0.  0.  0.  2.  3.]]


In [12]:
train_pickle_file = 'SVHN_basic_train_data_basic.pickle'

with open(train_pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  del save  # hint to help gc free up memory
  print('Training set',train_dataset.shape)
print(train_labels[0:2,:])

Training set (33402, 32, 32)
[[ 2.  0.  0.  0.  1.  9.]
 [ 2.  0.  0.  0.  2.  3.]]


In [13]:
test_pickle_file = 'SVHN_basic_test_labels.pickle'

with open(test_pickle_file, 'rb') as f:
  save = pickle.load(f)
  test_labels = save['test_image_labels'][0:1024]
  del save  # hint to help gc free up memory
  print('Test set', test_labels.shape)

test_pickle_file = 'SVHN_basic_test_data_basic.pickle'

with open(test_pickle_file, 'rb') as f:
  save = pickle.load(f)
  test_dataset = save['test_dataset'][0:1024]
  del save  # hint to help gc free up memory
  print('Test set', test_dataset.shape)

Test set (1024, 6)
Test set (1024, 32, 32)


####TODO: Randomize and Shuffle
####TODO: Use Boosting
####TODO: Use Simulated Annealing / Decay

###Reformat

In [14]:
### Reformat dataset to 1 channel
image_size=32
num_channels=1 #greyscale

def reformat(dataset):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels))\
        .astype(np.float32)
  return dataset

train_dataset = reformat(train_dataset)
test_dataset = reformat(test_dataset)
print("Reformatted shapes of datasets\n")
print("train_dataset.shape:",train_dataset.shape,", train_labels.shape:",train_labels.shape)
print("test_dataset.shape:", test_dataset.shape,", test_labels.shape:",test_labels.shape)

Reformatted shapes of datasets

train_dataset.shape: (33402, 32, 32, 1) , train_labels.shape: (33402, 6)
test_dataset.shape: (1024, 32, 32, 1) , test_labels.shape: (1024, 6)


### Define Accuracy

In [15]:
def accuracy(predictions, labels):
    #print("predictions:", predictions)
    #print("labels:", labels)
    label_count=labels.shape[0]
    len_preds=np.argmax(predictions[0,:,:],axis=1) - 4
    digit_1_preds=np.argmax(predictions[1,:,:],axis=1)
    digit_2_preds=np.argmax(predictions[2,:,:],axis=1)
    digit_3_preds=np.argmax(predictions[3,:,:],axis=1)
    digit_4_preds=np.argmax(predictions[4,:,:],axis=1)
    digit_5_preds=np.argmax(predictions[5,:,:],axis=1)
    #print("digit_1_preds:", digit_1_preds)
    #print("digit_2_preds:", digit_2_preds)
    #print("digit_3_preds:", digit_3_preds)
    #print("digit_4_preds:", digit_4_preds)
    #print("digit_5_preds:", digit_5_preds)

    
    len_accuracy=len_preds==labels[:,0]
    digit_1_accuracy=digit_1_preds==labels[:,1]
    digit_2_accuracy=digit_2_preds==labels[:,2]
    digit_3_accuracy=digit_3_preds==labels[:,3]
    digit_4_accuracy=digit_4_preds==labels[:,4]
    digit_5_accuracy=digit_5_preds==labels[:,5]
    
    
    complete_accuracy=np.concatenate((len_accuracy.reshape(1,label_count),digit_1_accuracy.reshape(1,label_count),\
                                      digit_2_accuracy.reshape(1,label_count),digit_3_accuracy.reshape(1,label_count), \
                                      digit_4_accuracy.reshape(1,label_count),digit_5_accuracy.reshape(1,label_count)), axis=0).T
    #print("complete_accuracy:\n",complete_accuracy.shape, complete_accuracy)
    return 100.0 * (np.sum([np.all(row) for row in complete_accuracy])) / len(labels)

#### Create Model

In [16]:
batch_size = 64
patch_size = 5
depth1 = 32
depth2 = 64
depth3 = 1024
num_hidden1 = 1024
num_hidden2 = 256
num_labels = 10 # 10 for 0-9
len_labels=6 #6 for lenghts0-5, not yet for more than 5
dropout = 0.50



graph = tf.Graph()

with graph.as_default():
  keep_prob = tf.placeholder(tf.float32)

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, len_labels))

  tf_digit_masks_1= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_2= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_3= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_4= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_digit_masks_5= tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  
  tf_test_dataset = tf.constant(test_dataset)

  
  # Variables.
  cnv_lyr1_weights = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, depth1], initializer=tf.contrib.layers.xavier_initializer_conv2d())
  cnv_lyr1_biases = tf.Variable(tf.constant(0.01, shape=[depth1]))

  cnv_lyr2_weights = tf.get_variable("W2", shape=[patch_size, patch_size, depth1, depth2], initializer=tf.contrib.layers.xavier_initializer_conv2d())
  cnv_lyr2_biases = tf.Variable(tf.constant(0.01, shape=[depth2]))

  cnv_lyr3_weights = tf.get_variable("W3", shape=[patch_size, patch_size, depth2, depth3], initializer=tf.contrib.layers.xavier_initializer_conv2d())
  cnv_lyr3_biases = tf.Variable(tf.constant(0.01, shape=[depth3]))
    
  fc_weights_1 = tf.get_variable("WFC1", shape=[depth2 * depth2 * depth3 // 64, num_hidden1], initializer=tf.contrib.layers.xavier_initializer())
  fc_biases_1 = tf.Variable(tf.constant(0.1, shape=[num_hidden1]))
    
  fc_weights_2 = tf.get_variable("WFC2", shape=[num_hidden1  , num_hidden2], initializer=tf.contrib.layers.xavier_initializer())
  fc_biases_2 = tf.Variable(tf.constant(0.1, shape=[num_hidden2]))

  length_weights = tf.get_variable("WL", shape=[num_hidden2 , len_labels], initializer=tf.contrib.layers.xavier_initializer())
  length_biases = tf.Variable(tf.constant(0.1, shape=[len_labels]))

  digit1_weights = tf.get_variable("WD1", shape=[num_hidden2 , num_labels], initializer=tf.contrib.layers.xavier_initializer())
  digit1_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  digit2_weights = tf.get_variable("WD2", shape=[num_hidden2 , num_labels],initializer=tf.contrib.layers.xavier_initializer())
  digit2_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  digit3_weights = tf.get_variable("WD3", shape=[num_hidden2 , num_labels], initializer=tf.contrib.layers.xavier_initializer())
  digit3_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  digit4_weights = tf.get_variable("WD4", shape=[num_hidden2, num_labels], initializer=tf.contrib.layers.xavier_initializer())
  digit4_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  digit5_weights = tf.get_variable("WD5", shape=[num_hidden2 , num_labels], initializer=tf.contrib.layers.xavier_initializer())
  digit5_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

  def max_pool_2x2(x):
     return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
  
  # Model.
  def model(data, digit_masks_1,digit_masks_2,digit_masks_3,digit_masks_4,digit_masks_5, is_train, dropout_prob):
    print("data.shape:",data.get_shape())
    
    conv = tf.nn.conv2d(data, cnv_lyr1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr1_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    #h_pool1 = max_pool_2x2(lrn)
    
    conv = tf.nn.conv2d(lrn, cnv_lyr2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr2_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    h_pool2 = max_pool_2x2(lrn)
    
    conv = tf.nn.conv2d(h_pool2, cnv_lyr3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + cnv_lyr3_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    h_pool3 = max_pool_2x2(lrn)
    
    shape = h_pool3.get_shape().as_list()
    conv_result_reshape = tf.reshape(h_pool3, [shape[0], shape[1] * shape[2] * shape[3]])
    print("conv_result_reshape:",conv_result_reshape.get_shape())
    hidden = tf.nn.relu(tf.matmul(conv_result_reshape, fc_weights_1) + fc_biases_1)
    hidden = tf.nn.relu(tf.matmul(hidden, fc_weights_2) + fc_biases_2)
    
    # Apply Dropout
    #hidden = tf.nn.dropout(hidden, dropout_prob)
    
    logit_length = tf.matmul(hidden, length_weights) + length_biases
    
    logit_1 = tf.nn.relu((tf.matmul(hidden, digit1_weights) + digit1_biases))
    #if (is_train == True):
    #    logit_1 = logit_1 * digit_masks_1
    
    logit_2 = tf.nn.relu((tf.matmul(hidden, digit2_weights) + digit2_biases))
    #if (is_train == True):
    #    logit_2 = logit_2 * digit_masks_2
        
    logit_3 = tf.nn.relu((tf.matmul(hidden, digit3_weights) + digit3_biases))
    #if (is_train == True):
    #    logit_3 = logit_3 * digit_masks_3

    logit_4 = tf.nn.relu((tf.matmul(hidden, digit4_weights) + digit4_biases))
    #if (is_train == True):
    #    logit_4 = logit_4 * digit_masks_4

    logit_5 = tf.nn.relu((tf.matmul(hidden, digit5_weights) + digit5_biases))
    #if (is_train == True):
    #    logit_5 = logit_5 * digit_masks_5

    

    return logit_length, logit_1,logit_2,logit_3,logit_4,logit_5
  
  # Training computation.
  logit_len, logit_digit_1,logit_digit_2,logit_digit_3,logit_digit_4,logit_digit_5 \
    = model(tf_train_dataset, tf_digit_masks_1, tf_digit_masks_2, tf_digit_masks_3, tf_digit_masks_4, \
            tf_digit_masks_5, True, keep_prob)

  loss_len     = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_len, tf_train_labels[:,0])
  loss_digit_1 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_1, tf_train_labels[:,1])
  loss_digit_2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_2, tf_train_labels[:,2])
  loss_digit_3 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_3, tf_train_labels[:,3])
  loss_digit_4 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_4, tf_train_labels[:,4])
  loss_digit_5 = tf.nn.sparse_softmax_cross_entropy_with_logits(logit_digit_5, tf_train_labels[:,5])
    
  loss = tf.reduce_mean(loss_len + loss_digit_1 + loss_digit_2 + loss_digit_3 + loss_digit_4 + loss_digit_5)
    
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.01
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  #optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
  
  # Predictions for TRAIN
  len_preds = tf.nn.softmax(logit_len)
  pads = tf.constant([[0, 0], [4, 0]])
  len_preds=tf.pad(len_preds, pads, "CONSTANT")
  train_prediction = tf.pack([ len_preds, tf.nn.softmax(logit_digit_1), tf.nn.softmax(logit_digit_2), tf.nn.softmax(logit_digit_3), \
                              tf.nn.softmax(logit_digit_4), tf.nn.softmax(logit_digit_5) ])

  #Predictions for TEST
  logit_len_test, logit_digit_1_test, logit_digit_2_test, logit_digit_3_test, logit_digit_4_test, logit_digit_5_test \
    = model(tf_test_dataset, tf_digit_masks_1, tf_digit_masks_2, tf_digit_masks_3, tf_digit_masks_4, tf_digit_masks_5, False, 1.)
  len_preds_test = tf.nn.softmax(logit_len_test)
  pads_test = tf.constant([[0, 0], [4, 0]])
  len_preds_test=tf.pad(len_preds_test, pads_test, "CONSTANT")
  test_prediction = tf.pack([ len_preds_test, tf.nn.softmax(logit_digit_1_test), tf.nn.softmax(logit_digit_2_test), \
                             tf.nn.softmax(logit_digit_3_test), tf.nn.softmax(logit_digit_4_test), \
                             tf.nn.softmax(logit_digit_5_test) ])
   
 

data.shape: (64, 32, 32, 1)
conv_result_reshape: (64, 65536)
data.shape: (1024, 32, 32, 1)
conv_result_reshape: (1024, 65536)


In [17]:
# Create 5 mask arrays; 1 for each digit.
# Each such Xth array will have all zeroes in Nth a row, if Xh digit is not present in Nth number in batch. 
def explode(batch_labels_arr):
    batch_size_count=batch_labels_arr.shape[0]
    target_col_count=10 # 10 labels for each digit
    total_digits_cols=5
    digit_mask_array = np.ndarray(shape=(total_digits_cols, batch_size_count, target_col_count),dtype=np.int32)
    digit_mask_array[:,:,:]=int(0)
    #print("batch_labels_arr: \n",batch_labels_arr)
    #print(digit_mask_array.shape)
    for row in range(batch_size_count):
        len_val=batch_labels_arr[row,0].astype(int)
        #print("len_val:",len_val)
        for counter in range(len_val):
            digit_mask_array[5-counter-1, row, :] = 1
    #print("digit_mask_array:\n",digit_mask_array)
    return digit_mask_array[0,:,:],digit_mask_array[1,:,:],digit_mask_array[2,:,:],digit_mask_array[3,:,:],digit_mask_array[4,:,:],
    
#print(train_labels[10:20,:])        
#explode(train_labels[10:20,:])

In [18]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :,:]
    batch_labels = train_labels[offset:(offset + batch_size), :].astype(int)
    digit_1_mask, digit_2_mask,digit_3_mask,digit_4_mask,digit_5_mask = explode(batch_labels)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_digit_masks_1 : digit_1_mask ,\
                tf_digit_masks_2 : digit_2_mask, tf_digit_masks_3 : digit_3_mask, tf_digit_masks_4 : digit_4_mask, \
                tf_digit_masks_5 : digit_5_mask, keep_prob : dropout}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step > 1 and step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
  

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 500: 8.339126
Minibatch accuracy: 0.0%
Minibatch loss at step 1000: 7.758300
Minibatch accuracy: 1.6%
Minibatch loss at step 1500: 4.627799
Minibatch accuracy: 3.1%
Minibatch loss at step 2000: 4.805098
Minibatch accuracy: 9.4%
Minibatch loss at step 2500: 4.196727
Minibatch accuracy: 28.1%
Minibatch loss at step 3000: 2.001112
Minibatch accuracy: 50.0%
Minibatch loss at step 3500: 2.949170
Minibatch accuracy: 42.2%
Minibatch loss at step 4000: 2.820929
Minibatch accuracy: 46.9%
Minibatch loss at step 4500: 2.501273
Minibatch accuracy: 43.8%
Minibatch loss at step 5000: 2.678669
Minibatch accuracy: 53.1%
Minibatch loss at step 5500: 2.084463
Minibatch accuracy: 50.0%
Minibatch loss at step 6000: 2.002605
Minibatch accuracy: 45.3%
Minibatch loss at step 6500: 1.659801
Minibatch accuracy: 64.1%
Minibatch loss at step 7000: 1.778015
Minibatch accuracy: 62.5%
Minibatch loss at step 7500: 1.019748
Minibatch accuracy: 73.4%
Minibatch loss at step 8000: 1.49

#### 